**Connect google drive**

In [1]:
# Check if NVIDIA GPU is activated to ensure faster training time
!nvidia-smi

Sun Dec  4 15:30:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'My Drive'   yolov3


**3) Configure Darknet**

In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.17 MiB | 15.19 MiB/s, done.
Resolving deltas: 100% (10405/10405), done.


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [8]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-12-04 15:32:12--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  21.0MB/s    in 8.4s    

2022-12-04 15:32:21 (18.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**Extract Images**


In [10]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/eyes_conjunctivitis2.jpg  
  inflating: data/obj/__MACOSX/._eyes_conjunctivitis2.jpg  
  inflating: data/obj/11493_2019_6011_Fig2_HTML.jpg  
  inflating: data/obj/__MACOSX/._11493_2019_6011_Fig2_HTML.jpg  
  inflating: data/obj/11493_2019_6011_Fig2_HTML.txt  
  inflating: data/obj/11493_2019_6011_Fig2_HTML.xml  
  inflating: data/obj/anemia-1-50-320.jpg  
  inflating: data/obj/__MACOSX/._anemia-1-50-320.jpg  
  inflating: data/obj/anemia-1-50-320.txt  
  inflating: data/obj/anemia-1-50-320.xml  
  inflating: data/obj/anemia.jpg     
  inflating: data/obj/__MACOSX/._anemia.jpg  
  inflating: data/obj/anemia.txt     
  inflating: data/obj/ca0911anemia_eyelid_195712.jpg  
  inflating: data/obj/__MACOSX/._ca0911anemia_eyelid_195712.jpg  
  inflating: data/obj/ca0911anemia_eyelid_195712.txt  
  inflating: data/obj/classes.txt    
  inflating: data/obj/T_1_20190606_092247_conjunctive_line_0.txt  
  inflating: data/obj/T_1_20190606_09

In [11]:
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.567187 0.337500 0.390625 0.162500

['15', '0.567187', '0.337500', '0.390625', '0.162500']
0 data/obj/Images-from-dataset-A-Non-anemic-patients-B-Anemic-patients-Color-figure-can-be_Q320.txt
0 0.567187 0.337500 0.390625 0.162500
15 0.571875 0.848437 0.381250 0.196875

['15', '0.571875', '0.848437', '0.381250', '0.196875']
0 data/obj/Images-from-dataset-A-Non-anemic-patients-B-Anemic-patients-Color-figure-can-be_Q320.txt
0 0.571875 0.848437 0.381250 0.196875
15 0.203911 0.709220 0.273743 0.226950

['15', '0.203911', '0.709220', '0.273743', '0.226950']
1 data/obj/images.txt
0 0.203911 0.709220 0.273743 0.226950
15 0.782123 0.670213 0.206704 0.163121

['15', '0.782123', '0.670213', '0.206704', '0.163121']
1 data/obj/images.txt
0 0.782123 0.670213 0.206704 0.163121
15 0.424479 0.818750 0.755208 0.362500

['15', '0.424479', '0.818750', '0.755208', '0.362500']
2 data/obj/T_1_20190606_092247_conjunctive_line_0.txt
0 0.424479 0.818750 0.755208 0.362500
15 0.252743 0.674589 0.196708 0.25959

In [12]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/anemia.jpg', 'data/obj/T_1_20190606_092247_conjunctive_line_0.jpg', 'data/obj/images.jpg', 'data/obj/eyes_conjunctivitis2.jpg', 'data/obj/ima22359-fig-0002-m.jpg', 'data/obj/Images-from-dataset-A-Non-anemic-patients-B-Anemic-patients-Color-figure-can-be_Q320.jpg', 'data/obj/11493_2019_6011_Fig2_HTML.jpg', 'data/obj/anemia-1-50-320.jpg', 'data/obj/ca0911anemia_eyelid_195712.jpg']


In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**Train the model**

In [14]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 602: 0.499383, 0.595222 avg loss, 0.000131 rate, 8.443677 seconds, 38528 images, 5.707393 hours left
Loaded: 0.000085 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.603046), count: 5, class_loss = 0.868725, iou_loss = 0.642541, total_loss = 1.511266 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.545586), count: 2, class_loss = 0.303360, iou_loss = 0.336871, total_loss = 0.640231 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000042, iou_loss = 0.000000, total_loss = 0.000042 
 total_bbox = 71252, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.704612), count: 7, class_loss = 0.349900, iou_loss = 0.596573, total_loss = 0.946473 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.517998), count: 2